<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Calibration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calibration calculation
This skript aims to allow the calculation and generation of a linear regression to allow the quantivication of phthalates in the environement.

nput: The the peak list files (classic structure, header = 6, by thermofisher) are uploaded to the directory titeled *calibration_input*, it is of key importance that the title of the files follows the format: Cal_"conc.nr"_"series.nr"_"identification.nr"_pl.csv Any changes to filename, path or other changes that might impact the file significantly need to be comented and marked extensively.

The following code serves to import the librarys and mount the google drive.

In [42]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import matplotlib.pyplot as plt
import os
import csv

# Mount Google Drive
drive.mount('/content/drive', force_remount = True)

# Set path to the folder containing the CSV files
path = '/content/drive/My Drive/calibration_input'

Mounted at /content/drive


This loop aims at importing the various files from the google drive, looping throug the colums and saving them in a new file.

In [43]:
# Outer loop to iterate over the number series
for n in range(4, 5):  # Adapted to the number of data sets (number of redundant analyses)
    files = glob.glob(f'/content/drive/My Drive/calibration_input/Cal_{n}ug*.csv')
    
    if not files:
        print(f'Files matching pattern "Cal_{n}ug*.csv" were not found.')
        break
    
    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)
        
        for i in range(num_rows):
            row_dfs = []
            for file in files:
                # Read in the CSV file
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/calibration_output/Cal_{n}ug_row{i+1}.csv', index=False, header=False)  # Corrected file name formatting and removed headers
            
            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)


4.35	4.31	4.39	65790142350.304	23.52	38623269390.338	25.48
6.21	6.17	6.23	22784087462.729	8.15	16633771663.194	10.97
7.74	7.7	7.77	31485384232.535	11.26	18383070244.296	12.13
13.33	13.29	13.4	43629929619.793	15.6	18042749107.326	11.9
18.65	18.6	18.69	33780756994.9	12.08	17930008588.157	11.83
20.44	20.39	20.54	39061674749.547	13.97	20829965069.204	13.74
22.02	21.98	22.23	43143557380.228	15.43	21146814279.364	13.95


The following lines use the previously created row file and take the average and the standard deviation *(or the error)* and add them to two new files.

[]


In [45]:
# Get a list of all CSV files in the current directory
files = glob.glob('/content/drive/My Drive/calibration_output/Cal_*.csv')

# Loop over each file
for file in files:
    # Load the CSV file into a dataframe
    df = pd.read_csv(file, header=None)
    
    # Calculate the mean and standard deviation
    df_mean = df.mean(axis=0)
    df_mean = df_mean.transpose().to_frame().T
    df_std = df.std(axis=0)
    df_std = df_std.transpose().to_frame().T

    print(df_mean)
    print(df_std)
    
    # Get the file name without the directory path
    file_name = os.path.basename(file)
    
    # Save the mean dataframe to a new CSV file
    mean_file_path = '/content/drive/My Drive/calibration_output/' + file_name[:-4] + '_mean.csv'
    df_mean.to_csv(mean_file_path, index=False)
    
    # Save the standard deviation dataframe to a new CSV file
    std_file_path = '/content/drive/My Drive/calibration_output/' + file_name[:-4] + '_std.csv'
    df_std.to_csv(std_file_path, index=False)


        0      1       2             3        4             5        6
0  5.0525  5.015  5.1025  8.495416e+10  23.4075  4.227881e+10  22.0775
          0         1         2             3         4             5        6
0  1.398341  1.396698  1.411698  2.166718e+10  0.379594  8.012253e+09  4.88897
      0     1     2             3       4             5       6
0  7.31  7.28  7.34  2.609008e+10  7.3675  1.811311e+10  9.4675
          0         1         2             3         4             5  \
0  2.193338  2.200015  2.200015  3.577743e+09  0.949469  1.721503e+09   

          6  
0  1.790054  
        0     1     2             3       4             5        6
0  8.8825  8.84  8.92  3.922769e+10  10.885  2.512585e+10  12.5425
          0     1         2             3         4             5         6
0  2.278338  2.28  2.280015  8.644930e+09  0.483908  7.804304e+09  0.639134
        0        1      2             3      4             5      6
0  14.055  14.0075  14.11  5.681170e+10  15

Two files contain the internal standard signals are used to take the ratio of the various signals and save the values in a new file. The file needs to be specified so that the internal standard signal correlates to the divisor name, so that this leads to the correct ratio. The first one calculates the ratio of the mean and the bottom one being used for the ratio of the standard deviation, although that might be insuficient and should be substituted by an appropriate error calculation (possibly gaussian errorpropagation).

In [ ]:
# Read divisor value from the divisor file
divisor_file = 'divisor.csv'  # Replace with the name of your divisor file

with open(divisor_file, 'r') as file:
    reader = csv.reader(file)
    divisor = next(reader)[0]  # Assuming the divisor is in the first column

# Get a list of CSV files in the directory
directory = '/content/drive/My Drive/calibration_output'  # Replace with the actual path to the directory containing the CSV files

csv_files = glob.glob(os.path.join(directory, '*_mean.csv'))

# Process each CSV file
for csv_file in csv_files:
    if csv_file == divisor_file:
        continue  # Skip the divisor file itself

    # Read the contents of the CSV file
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)

    # Perform division on the data
    divided_data = [[str(float(value) / float(divisor)) for value in row] for row in data]

    # Create a new file name with the "_ratio" tag
    new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv' #needs to be adapted to fit the final data file name formate

    # Write the divided data to the new CSV file
     with open(new_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(divided_data)

In [ ]:
# Read divisor value from the divisor file
divisor_file = 'divisor.csv'  # Replace with the name of your divisor file

with open(divisor_file, 'r') as file:
    reader = csv.reader(file)
    divisor = next(reader)[0]  # Assuming the divisor is in the first column

# Get a list of CSV files in the directory
directory = '/content/drive/My Drive/calibration_output'  # Replace with the actual path to the directory containing the CSV files

csv_files = glob.glob(os.path.join(directory, '*_std.csv'))

# Process each CSV file
for csv_file in csv_files:
    if csv_file == divisor_file:
        continue  # Skip the divisor file itself

    # Read the contents of the CSV file
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)

    # Perform division on the data
    divided_data = [[str(float(value) / float(divisor)) for value in row] for row in data]

    # Create a new file name with the "_ratio" tag
    new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv' #needs to be adapted to fit the final data file name formate

    # Write the divided data to the new CSV file
     with open(new_file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(divided_data)

The following script creates a plot using the previously optained data. A file containing the concentration will be nescessary to associate the y-axis values with the x-axis values.